In [ ]:
import pandas as pd
import os
import numpy as np

### Carregamento e Inspeção Inicial dos Dados

In [ ]:
# Carregar o dataset
df = pd.read_csv('**local data***/dados_marketing.csv', sep=';')

# Verificar nulos
print(df.isnull().sum())

### Tratamento de Valores Nulos

In [ ]:
# Análise dos registros com 'Salario Anual' nulo
salarios_nulos = df[df['Salario Anual'].isnull()]
print(pd.DataFrame(salarios_nulos['ID']).rename(columns={'ID': 'ID com Salário Nulo'}))

In [ ]:
# Remoção dos nulos
df_inicial = df.shape[0]
df = df.dropna(subset=['Salario Anual'])
df_sem_nulos = df.shape[0]
print(f'Foram removidas {df_inicial - df_sem_nulos} linhas com salário nulo.')

### Tratamento de Duplicatas

In [ ]:
# Remoção de duplicatas
duplicatas = df[df.duplicated()]
soma_duplicadas = df.duplicated().sum()
df = df.drop_duplicates()
print(f'Foram removidas {soma_duplicadas} linhas duplicadas.')

### Engenharia de Atributos

In [ ]:
# Conversão de datas
df['Data Cadastro'] = pd.to_datetime(df['Data Cadastro'], dayfirst=True)

In [ ]:
# Criação de atributos derivados da Data de Cadastro
df['Ano Cadastro'] = df['Data Cadastro'].dt.year
df['Mes Cadastro'] = df['Data Cadastro'].dt.month
df['Mes Nome Cadastro'] = df['Data Cadastro'].dt.month_name()
df['Dia Cadastro'] = df['Data Cadastro'].dt.day
df['Dia Semana Cadastro'] = df['Data Cadastro'].dt.day_name()
df['Semana Cadastro'] = df['Data Cadastro'].dt.isocalendar().week

df['Idade'] = 2025 - df['Ano Nascimento']

In [ ]:
# Criação de atributos de segmentação de idade
df['Faixa Etária'] = pd.cut(df['Idade'], bins=[17,30,45,60,100], labels=['18-30', '31-45', '46-60', '60+'])

In [ ]:
# Criação de atributo binário para presença ou não de filhos/adolescentes
df['Filhos'] = np.where(df['Filhos em Casa'] > 0, 1, 0) | np.where(df['Adolescentes em Casa'] > 0, 1, 0)

### Criação de Métricas Derivadas

In [ ]:
# Criação de métricas derivadas de gastos
gastos = ['Gasto com Eletronicos', 'Gasto com Brinquedos', 'Gasto com Moveis',
          'Gasto com Utilidades', 'Gasto com Alimentos', 'Gasto com Vestuario']
df['Gasto Total'] = df[gastos].sum(axis=1)
compras = ['Numero de Compras com Desconto', 'Numero de Compras na Web',
           'Numero de Compras via Catalogo', 'Numero de Compras na Loja']
df['Total Compras'] = df[compras].sum(axis=1)

In [ ]:
# Criação de métricas derivadas de campanhas
df['Comprou em Campanha'] = np.where(df[[c for c in df.columns if 'campanha' in c.lower()]].sum(axis=1) > 0, 1, 0)
df['Engajamento em campanhas'] = df[[c for c in df.columns if 'campanha' in c.lower()]].sum(axis=1)

In [ ]:
# Descrição do dataframe após a inclusão de novas colunas
df.info()

### Padronização e Conversão de Tipos de Dados

In [ ]:
# Função para padronizar tipos de dados
def padronizar_tipos_numericos(df):
    # Colunas que devem ser float64
    colunas_float = [
        'Salario Anual',
        'Gasto com Eletronicos',
        'Gasto com Brinquedos',
        'Gasto com Moveis',
        'Gasto com Utilidades',
        'Gasto com Alimentos',
        'Gasto com Vestuario',
        'Gasto Total'
    ]

    # Colunas que devem ser int64
    colunas_int = [
        'ID',
        'Ano Nascimento',
        'Filhos em Casa',
        'Adolescentes em Casa',
        'Dias Desde Ultima Compra',
        'Numero de Compras com Desconto',
        'Numero de Compras na Web',
        'Numero de Compras via Catalogo',
        'Numero de Compras na Loja',
        'Numero Visitas WebSite Mes',
        'Compra na Campanha 1',
        'Compra na Campanha 2',
        'Compra na Campanha 3',
        'Compra na Campanha 4',
        'Compra na Campanha 5',
        'Comprou',
        'Ano Cadastro',
        'Mes Cadastro',
        'Dia Cadastro',
        'Semana Cadastro',
        'Idade',
        'Filhos',
        'Total Compras',
        'Comprou em Campanha',
        'Engajamento em campanhas'
    ]

    # Aplica conversão
    for col in colunas_float:
        df[col] = df[col].astype('float64')

    for col in colunas_int:
        df[col] = df[col].astype('int64')

    return df


In [ ]:
# Chama a função para efetivar a padronização
df = padronizar_tipos_numericos(df)

In [ ]:
# Conversão de colunas binárias (valores 0 e 1) para Sim/Não
colunas_binarias = [col for col in df.columns
                    if df[col].dtype == 'int64' and set(df[col].unique()) <= {0, 1}]

# Converte para Sim/Não
for col in colunas_binarias:
    df[col] = df[col].map({1: 'Sim', 0: 'Não'})

### Padronização de Nomes de Colunas

In [ ]:
# Padronização de nomes de colunas
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('ã', 'a').str.replace('ç', 'c').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [ ]:
# Verificação da padronização de colunas
print(df.columns.tolist())

### Tradução de Dias e Meses (inglês > português)

In [ ]:
# Mapeamento de dias da semana do inglês para português
dias_semana = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

# Aplica a tradução
df['dia_semana_cadastro'] = df['dia_semana_cadastro'].map(dias_semana)

In [ ]:
# Verificar se a tradução foi devidamente aplicada
print(df['dia_semana_cadastro'].unique())

In [ ]:
# Mapeamento de meses do ano do inglês para português
meses_ano = {
    'January': 'Janeiro',
    'February': 'Fevereiro',
    'March': 'Março',
    'April': 'Abril',
    'May': 'Maio',
    'June': 'Junho',
    'July': 'Julho',
    'August': 'Agosto',
    'September': 'Setembro',
    'October': 'Outubro',
    'November': 'Novembro',
    'December': 'Dezembro'
}

# Aplica a tradução
df['mes_nome_cadastro'] = df['mes_nome_cadastro'].map(meses_ano)

In [ ]:
# Verificar se a tradução foi devidamente aplicada
print(df['mes_nome_cadastro'].unique())

### Checagem final das colunas

In [ ]:
for col in df.columns:
    print(f"\n🔹 Coluna: {col}")
    valores = df[col].unique()
    print(valores if len(valores) <= 20 else f"{len(valores)} valores únicos. Exibindo os 20 primeiros:")
    print(valores[:20])

### Estatísticas Básicas

In [ ]:
# Seleciona colunas numéricas, excluindo 'id' e datas
colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
colunas_numericas = [col for col in colunas_numericas if col != 'id']

# Exibe estatísticas básicas
print(df[colunas_numericas].describe())

### Detecção de Outliers

In [ ]:
outliers = {}

for col in colunas_numericas:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Identifica outliers
    outlier_mask = (df[col] < limite_inferior) | (df[col] > limite_superior)
    outliers[col] = df[col][outlier_mask]

    print(f"\nColuna: {col}")
    print(f"Outliers detectados: {outlier_mask.sum()}")
    print(f"Valores extremos: {df[col][outlier_mask].unique()}")

### Tratamento de Outliers

In [ ]:
# Localiza registros com ano de nascimento incoerente
df_outliers_nascimento = df[df['ano_nascimento'] < 1920]
print(df_outliers_nascimento[['id', 'idade', 'ano_nascimento']])

In [ ]:
# Remoção dos registros de idade incoerente
df_sem_nulos = df.shape[0]
df = df.drop(df_outliers_nascimento.index)
df_sem_ano_incoerente = df.shape[0]
print(f'Foram removidas {df_sem_nulos - df_sem_ano_incoerente} linhas com idade incoerente.')

### Salvamento do Dataset Limpo

In [ ]:
print(f'Tamanho inicial do DataFrame: {df_inicial} linhas')
print(f'Tamanho final do DataFrame: {df.shape[0]} linhas')
print(f'Total de linhas removidas: {df_inicial - df.shape[0]} linhas')

porcentagem = (df_inicial - df.shape[0]) / df_inicial * 100
print(f'Porcentagem de linhas removidas: {porcentagem: .2f} %')

In [ ]:
# Salvamento condicional
output_path = '***local data***/dados_marketing_limpo.csv'
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df.to_csv(output_path, index=False, encoding='utf-8')
print(f'Dataset limpo salvo em: {output_path}')